# MDG NAP Preliminary Analysis

In [195]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from scipy import stats

init_notebook_mode(connected=True)

engine = create_engine('postgresql://postgres@localhost:5433/cartodb_user_fddc7a0f-c829-4ec0-a4b8-212ec7ce9ad1_db')
sql_rast_mean_by_region = """
SELECT {group_by},
(ST_SummaryStatsAgg(ST_Clip(the_raster_webmercator, the_geom_webmercator, {na_value}, true), 1, true)).* 
FROM {raster}, {poly}
WHERE ST_Intersects(the_raster_webmercator, the_geom_webmercator)
GROUP BY {group_by};
"""

# General function to compare series
def plot_comparison(x, y, labels, x_label, y_label, size=None, trend=True,
                    title=None, color=None, colorbartitle='Colorbar'):
    if color is not None:
        trace = go.Scatter(
            x = x,
            y = y,
            text = labels,
            mode = 'markers',
            showlegend = False,
            marker = dict(
                sizemode = 'area',
                size = size,
                color = color,
                colorbar = go.ColorBar(
                    title = colorbartitle
                ),
                colorscale='Viridis'
            )
        )
    else:
        trace = go.Scatter(
            x = x,
            y = y,
            text = labels,
            mode = 'markers',
            showlegend = False,
            marker = dict(
                sizemode = 'area',
                size = size,
            )
        )
    if trend:
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
        trendline = slope * x + intercept
        r, p = stats.pearsonr(x, y)
        trend_trace = go.Scatter(
                  x=x, 
                  y=trendline, 
                  mode='lines',
                  opacity = 0.5,
                  showlegend = False,
                  hoverinfo='none'
        )
        annotation = go.Annotation(
                  x=.9,
                  y=.9,
                  xref = 'paper',
                  yref = 'paper',
                  text = 'r = {:.2}, p = {:.2}'.format(r, p),
                  showarrow=False,
                  font=go.Font(size=16, family='Arial, italic')
        )
        layout = go.Layout(
            title=title,
            xaxis=dict(title=x_label),
            yaxis=dict(title=y_label),
            annotations = [annotation],
            showlegend = False
        )
        data = [trace, trend_trace]
    else:
        layout = go.Layout(
            title=title,
            xaxis=dict(title=x_label),
            yaxis=dict(title=y_label)
        )
        data = [trace]
    fig = go.Figure(data=data, layout=layout)
    return(iplot(fig, filename='basic-scatter'))

# General function to compare two rasters by adm2 region
def get_rast_mean_by_region(rast, region="(select * from gadm28_adm2 where iso in ('MDG')) as regions"):
    key = {'raster': rast[1], # This is the raster table name
           'poly': region,
           'na_value': -9999,
           'group_by': 'name_2'}
    df = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
    df['variable'] = rast[0] # This is a short name for the raster
    return(df)

## Agriculture / water

### Climate trends

In [2]:
# Pull precipitation trends
key = {'raster': 'total_annual_rainfall_multimodel_pctdiff_mean_85_50_mdg',
       'poly': "(select * from gadm28_adm2 where iso='MDG') as polys",
       'na_value': -9999,
       'group_by': 'name_2'}
df_rain = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_rain = df_rain.rename(columns={'mean':'precip'})

# Pull temperature trends
key = {'raster': 'temp_diff_multimodel_mean_4550',
       'poly': "(select * from gadm28_adm2 where iso='MDG') as polys",
       'na_value': -9999,
       'group_by': 'name_2'}
df_temp = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_temp = df_temp.rename(columns={'mean':'temp'})
df_temp.temp = df_temp.temp / 10

# Pull population
key = {'raster': 'lspop2014',
       'poly': "(select * from gadm28_adm2 where iso='MDG') as polys",
       'na_value': -9999,
       'group_by': 'name_2'}
df_pop = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_pop = df_pop.rename(columns={'sum':'pop'})

# Merge datasets
df_clim_districts = pd.merge(df_rain[['name_2', 'precip']], df_temp[['name_2', 'temp']])
df_clim_districts = pd.merge(df_clim_districts, df_pop[['name_2', 'pop']])
df_clim_districts = df_clim_districts.set_index('name_2')

In [170]:
plot_comparison(df_clim_districts.temp, df_clim_districts.precip, df_clim_districts.index,
                x_label='Temperature change (°C)', y_label='Precipitation change (%)')

### Health versus climate

In [106]:
# Pull precipitation trends
key = {'raster': 'total_annual_rainfall_multimodel_pctdiff_mean_85_50_mdg',
       'poly': 'sdr_subnational_data_dhs_2008',
       'na_value': -9999,
       'group_by': 'dhsregen'}
df_rain = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_rain = df_rain.rename(columns={'mean':'precip'})

# Pull temperature trends
key = {'raster': 'temp_diff_multimodel_mean_4550',
       'poly': 'sdr_subnational_data_dhs_2008',
       'na_value': -9999,
       'group_by': 'dhsregen'}
df_temp = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_temp = df_temp.rename(columns={'mean':'temp'})
df_temp.temp = df_temp.temp / 10

# Pull population
key = {'raster': 'lspop2014',
       'poly': 'sdr_subnational_data_dhs_2008',
       'na_value': -9999,
       'group_by': 'dhsregen'}
df_pop = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_pop = df_pop.rename(columns={'sum':'pop'})

# Merge both climate datasets
df_clim_dhs = pd.merge(df_rain[['dhsregen', 'precip']], df_temp[['dhsregen', 'temp']])
df_clim_dhs = pd.merge(df_clim_dhs, df_pop[['dhsregen', 'pop']])

# Pull health data
df_dhs = pd.read_sql_query('select cnnutscha2 as stunting, cmecmrcimr as imr, mlitnapacc as pct_access_nets, dhsregen from sdr_subnational_data_dhs_2008', con=engine)
df_dhs = pd.merge(df_clim_dhs, df_dhs)
df_dhs = df_dhs.set_index('dhsregen')
df_dhs.size_by_pop = (df_dhs['pop'] / df_dhs['pop'].max()) * 1000

#### Stunting

In [52]:
plot_comparison(df_dhs['stunting'], df_dhs.precip, df_dhs.index,
                x_label='Children that are stunted (%)', y_label='Precipitation change (%)',
                size=df_dhs.size_by_pop, trend=True)

In [6]:
plot_comparison(df_dhs['stunting'], df_dhs.temp, df_dhs.index,
                x_label='Children that are stunted (%)', y_label='Temperature change (°C)',
                size=df_dhs.size_by_pop)

#### Infant mortality rates

In [7]:
plot_comparison(df_dhs['imr'], df_dhs.precip, df_dhs.index,
                x_label='Infant mortality rate', y_label='Precipitation change (%)',
                size=df_dhs.size_by_pop)

In [8]:
plot_comparison(df_dhs['imr'], df_dhs.temp, df_dhs.index,
                x_label='Infant mortality rate', y_label='Temperature change (°C)',
                size=df_dhs.size_by_pop)

#### Access to bed nets

In [9]:
plot_comparison(df_dhs['pct_access_nets'], df_dhs.precip, df_dhs.index,
                x_label='Percentage with access to insecticide treated nets', y_label='Precipitation change (%)',
                size=df_dhs.size_by_pop)

In [10]:
plot_comparison(df_dhs['pct_access_nets'], df_dhs.temp, df_dhs.index,
                x_label='Percentage with access to insecticide treated nets', y_label='Temperature change (°C)',
                size=df_dhs.size_by_pop)

#### Climate and food security - IGNORE FOR NOW, NEED TO REVIEW DATA

In [11]:
# Pull precipitation trends
key = {'raster': 'total_annual_rainfall_multimodel_pctdiff_mean_85_50_mdg',
       'poly': 'communes_poverty_fid_2007',
       'na_value': -9999,
       'group_by': 'first_nom_'}
df_rain = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_rain = df_rain.rename(columns={'mean':'precip'})

# Pull temperature trends
key = {'raster': 'temp_diff_multimodel_mean_4550',
       'poly': 'communes_poverty_fid_2007',
       'na_value': -9999,
       'group_by': 'first_nom_'}
df_temp = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_temp = df_temp.rename(columns={'mean':'temp'})
df_temp.temp = df_temp.temp / 10

# Pull population
key = {'raster': 'lspop2014',
       'poly': 'communes_poverty_fid_2007',
       'na_value': -9999,
       'group_by': 'first_nom_'}
df_pop = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_pop = df_pop.rename(columns={'sum':'pop'})

# Merge climate and population datasets
df_clim_commune = pd.merge(df_rain[['first_nom_', 'precip']], df_temp[['first_nom_', 'temp']])
df_clim_commune = pd.merge(df_clim_commune, df_pop[['first_nom_', 'pop']])

# Pull health data
df_commune = pd.read_sql_query('select poor_per as food_insecurity, first_nom_ from communes_poverty_fid_2007', con=engine)
df_commune = pd.merge(df_clim_commune, df_commune)
df_commune = df_commune.set_index('first_nom_')
df_commune.size_by_pop = (df_commune['pop'] / df_commune['pop'].max()) * 1000

In [12]:
plot_comparison(df_commune['food_insecurity'], df_commune['precip'], df_commune.index,
                x_label='Percentage food insecure in ??', y_label='Precipitation change (%)',
                size=df_commune.size_by_pop)

In [13]:
plot_comparison(df_commune['food_insecurity'], df_commune['temp'], df_commune.index,
                x_label='Percentage food insecure in ??', y_label='Temperature change (°C)',
                size=df_commune.size_by_pop)

#### Food security versus health stress - HOLD OFF UNTIL WE REVIEW THE FOOD INSECURITY LAYER

#### Climate versus rice production and essential natural capital for food security

Look at correlations among climate change, nutrition, and rice production:
- Calculate production per district
- Normalize production by district-level population to get per-capita production
- Plot versus climate change and nutrition indicators, sizing bubbles by per-capita production


In [108]:
# Pull rice production data and join it with district-level climate chagne
df_fs = pd.read_sql_query('select rc_ha_10 as rice_prod, region as name_2 from mdg_agriculture_productionbyregion_20150413', con=engine)
df_fs = pd.merge(df_clim_districts, df_fs, left_index=True, right_on='name_2')
df_fs = df_fs.set_index('name_2')
df_fs['rice_prod_per_capita'] = df_fs.rice_prod / df_fs['pop']
df_fs['size_percapita_rice_prod'] = (df_fs['rice_prod_per_capita'] / df_fs['rice_prod_per_capita'].max()) * 1000


In [49]:
plot_comparison(df_fs.temp, df_fs.precip, df_fs.index,
                x_label='Temperature change (°C)', y_label='Precipitation change (%)',
                size=df_fs.size_percapita_rice_prod)

Look at climate trends in areas of natural capital important for rice production compared to climate trends in rest of Madagascar:

In [112]:
# Join the DHS-derived indicators
df_fs_dhs = pd.merge(df_fs, df_dhs.drop(['precip', 'temp', 'pop'], axis=1),
                     left_index=True, right_index=True)

In [192]:
plot_comparison(df_fs_dhs.temp, df_fs_dhs.precip, df_fs_dhs.index,
                x_label='Temperature change (°C)', y_label='Precipitation change (%)',
                size=df_fs_dhs.stunting*10,
                color=df_flood.stunting,
                colorbartitle='Stunting')

## Disaster risk reduction

In [201]:
# Calculate change in flood frequency with climate change by district
key = {'raster': 'rcp85_100yrflood_yr2100_returnperiod',
       'poly': "(select * from gadm28_adm2 where iso='MDG') as polys",
       'na_value': -9999,
       'group_by': 'name_2'}
df_flood = pd.read_sql_query(sql_rast_mean_by_region.format(**key), con=engine)
df_flood = df_flood.rename(columns={'mean':'flood_2050'})
df_flood = df_flood[['name_2', 'flood_2050']]
df_flood = df_flood.set_index('name_2')

# Add area natural capital important for rice production
area_important_by_region = """
SELECT {group_by},
(select sum(hectares) from ST_Union(ST_Intersection(regions.the_geom_webmercator, polys.the_geom_webmercator))) as total
FROM {regions} as regions, {polys} as polys
WHERE ST_Intersects(regions.the_geom_webmercator, polys.the_geom_webmercator)
GROUP BY {group_by};
"""
key = {'regions': "(select * from gadm28_adm2 where iso='MDG')",
       'polys': "(select * from ricethrespoly)",
       'group_by': 'name_2'}
df_natcap_rice = pd.read_sql_query(area_important_by_region.format(**key), con=engine)
df_natcap_rice = df_natcap_rice.rename(columns={'total':'natcap_imp_rice_hectares'})
df_natcap_rice = df_natcap_rice.set_index('name_2')

# Join the DHS-derived indicators
df_flood = pd.merge(df_fs_dhs, df_flood, left_index=True, right_index=True)
df_flood = pd.merge(df_flood, df_natcap_rice, left_index=True, right_index=True)

#### Flooding versus climate, health, and rice production

In [202]:
plot_comparison(df_flood.flood_2050, df_flood.precip, df_flood.index,
                x_label='100 year flood return interval in 2100 (in years)',
                y_label='Precipitation change (%)',
                size=df_flood.stunting*10,
                color=df_flood.stunting,
                colorbartitle='Stunting')

In [203]:
plot_comparison(df_flood.flood_2050, df_flood.precip, df_flood.index,
                x_label='100 year flood return interval in 2100 (in years)',
                y_label='Precipitation change (%)',
                size=df_flood.imr*10,
                color=df_flood.imr,
                colorbartitle='Infant mortality rate')

In [204]:
plot_comparison(df_flood.flood_2050, df_flood.precip, df_flood.index,
                x_label='100 year flood return interval in 2100 (in years)',
                y_label='Precipitation change (%)',
                size=df_flood.pct_access_nets*10,
                color=df_flood.pct_access_nets,
                colorbartitle='Percent with access to bednets')

In [205]:
plot_comparison(df_flood.flood_2050, df_flood.precip, df_flood.index,
                x_label='100 year flood return interval in 2100 (in years)',
                y_label='Precipitation change (%)',
                size=df_flood.rice_prod_per_capita*1000,
                color=df_flood.rice_prod_per_capita,
                colorbartitle='Rice production (per capita)')

#### Flooding versus rice production and natural capital area important for rice production

In [209]:
plot_comparison(df_flood.flood_2050, df_flood.rice_prod_per_capita, df_flood.index,
                x_label='100 year flood return interval in 2100 (in years)',
                y_label='Rice production (per capita)',
                size=df_flood.natcap_imp_rice_hectares/1000,
                color=df_flood.natcap_imp_rice_hectares,
                colorbartitle='Nat. cap. imp. for rice prod. (hectares)')

In [ ]:
Make connection to slide 11, with landscan and mangroves to social
Add freshwater for irrigation
DRR with flood and drought

Slide 11 similar for mangroves

Flooding by district, with relative important and also 

Inland and marine fisheries
Social layers as well
Send papers
Get up project list

In [ ]:
# Areas important for rice irrigation


#### Coral and mangroves versus coastal vulnerability

## Relative exposure of natural capital to climate chage